In [1]:
'''>>>>> start CodeP2.2
    V.P. Carey ME249, Spring 2021

Intro to Neural Network Modeling 
Keras model for comparison with first principles model'''

#import useful packages
import keras
import pandas as pd
from keras.models import Sequential
import numpy as np
import keras.backend as kb
import tensorflow as tf

#the follwoing 2 lines are only needed for Mac OS machines
# import os
# os.environ['KMP_DUPLICATE_LIB_OK']='True'

#raw data in dictionary form x01, x02, x03, y3
my_dict = { 
    'x01' : [20., 20., 20., 20.2, 20., 20.2, 24.0, 36.],
    'x02' : [13., 14.5, 15.3, 13., 14.5, 15.3, 13., 14.5],
    'x03' : [310.8, 308.0, 306.0, 310.8, 308.0, 306.0, 310.8, 308.0],
    'y3' : [30.97, 32.3, 31.5, 30.91, 32.5, 31.4, 35.59, 46.4]
}
#normalized inputs in array
xdata = []
xdata = [[20./20.2, 13.0/14.5, 310.8/308.0], [20./20.2, 14.5/14.5, 308.0/308.0]] 
xdata.append([20./20.2, 15.3/14.5, 306.0/308.0])
xdata.append([20.2/20.2, 13.0/14.5, 310.8/308.0]) 
xdata.append([20./20.2, 14.5/14.5, 308.0/308.0]) 
xdata.append([20.2/20.2, 15.3/14.5, 306.0/308.0]) 
xdata.append([24./20.2, 13.0/14.5, 310.8/308.0]) 
xdata.append([36./20.2, 14.5/14.5, 308.0/308.0]) 
#data frame
df = pd.DataFrame(my_dict)
#devide by the median to normalize 
df.x01= df.x01/20.2
df.x02= df.x02/14.5
df.x03= df.x03/308.0
#normalize output array
df.y3= df.y3/32.401
df.head
print (df.x01, df.x02, df.x03, df.y3)

xarray= np.array(xdata)
print (xdata)
print (xarray)

Using TensorFlow backend.


0    0.990099
1    0.990099
2    0.990099
3    1.000000
4    0.990099
5    1.000000
6    1.188119
7    1.782178
Name: x01, dtype: float64 0    0.896552
1    1.000000
2    1.055172
3    0.896552
4    1.000000
5    1.055172
6    0.896552
7    1.000000
Name: x02, dtype: float64 0    1.009091
1    1.000000
2    0.993506
3    1.009091
4    1.000000
5    0.993506
6    1.009091
7    1.000000
Name: x03, dtype: float64 0    0.955835
1    0.996883
2    0.972192
3    0.953983
4    1.003055
5    0.969106
6    1.098423
7    1.432055
Name: y3, dtype: float64
[[0.9900990099009901, 0.896551724137931, 1.009090909090909], [0.9900990099009901, 1.0, 1.0], [0.9900990099009901, 1.0551724137931036, 0.9935064935064936], [1.0, 0.896551724137931, 1.009090909090909], [0.9900990099009901, 1.0, 1.0], [1.0, 1.0551724137931036, 0.9935064935064936], [1.188118811881188, 0.896551724137931, 1.009090909090909], [1.7821782178217822, 1.0, 1.0]]
[[0.99009901 0.89655172 1.00909091]
 [0.99009901 1.         1.        ]
 [0.990

In [2]:
# define model

#As seen below, we have created three dense layers each with just one neuron. 
#A dense layer is a layer in neural network that’s fully connected. 
#In other words, all the neurons in one layer are connected to all other neurons in the next layer.
#In the first layer, we need to provide the input shape, which is 3 in this case. 
#The activation function we have chosen is ReLU, which stands for rectified linear unit.

from keras import backend as K
#initialize weights with values between -0.2 and 1.2
initializer = tf.keras.initializers.RandomUniform(minval= -0.2, maxval=1.2)

# define three layer model with one neuron in each layer
model = keras.Sequential([
    keras.layers.Dense(1, activation=K.elu, input_shape=[3]),
    keras.layers.Dense(1, activation=K.elu),
    keras.layers.Dense(1, activation=K.elu)
  ])


In [3]:
# Set weights in the input layer
model.layers[0].set_weights([np.array([[1.8],[0.55],[1]]),np.array([-0.2])])

In [4]:
# Set weights in first hidden layer
model.layers[1].set_weights([np.array([[0.9]]),np.array([-0.07])])

In [5]:
# Set weights in output layer
model.layers[2].set_weights([np.array([[0.4]]),np.array([0.05])])

In [6]:
#We’re using RMSprop as our optimizer here. RMSprop stands for Root Mean Square Propagation. 
#It’s one of the most popular gradient descent optimization algorithms for deep learning networks. 
#RMSprop is an optimizer that’s reliable and fast.
#We’re compiling the mode using the model.compile function. The loss function used here 
#is mean absolute error. After the compilation of the model, we’ll use the fit method with 100 epochs.

#Running model.fit successive times extends the calculation to addtional epochs.

sgd = keras.optimizers.RMSprop(learning_rate=0.1)
model.compile(loss='mean_absolute_error',optimizer=sgd)


In [7]:
#After the compilation of the model, we’ll use the fit method with 500 epochs.
#I started with epochs value of 100 and then tested the model after training. 
#The prediction was not that good. Then I modified the number of epochs to 200 and tested the model again. 
#Accuracy had improved slightly, but figured I’d give it one more try. Finally, at 500 epochs 
#I found acceptable prediction accuracy.

#The fit method takes three parameters; namely, x, y, and number of epochs. 
#During model training, if all the batches of data are seen by the model once, 
#we say that one epoch has been completed.

# Add an early stopping callback
es = keras.callbacks.EarlyStopping(
    monitor='loss', 
    mode='min', 
    patience = 20, 
    restore_best_weights = True, 
    verbose=1)
# Add a checkpoint where loss is minimum, and save that model
mc = keras.callbacks.ModelCheckpoint('best_model.SB', monitor='loss', 
                     mode='min',  verbose=1, save_best_only=True)

In [8]:
historyData = model.fit(xarray,df.y3,epochs=100,callbacks=[es,mc])

loss_hist = historyData.history['loss']
#The above line will return a dictionary, access it's info like this:
best_epoch = np.argmin(historyData.history['loss']) + 1
print ('best epoch = ', best_epoch)
print('smallest loss =', np.min(loss_hist))

Epoch 1/100
8/8 [==============================] - 1s 130ms/step - loss: 0.1794

Epoch 00001: loss improved from inf to 0.17939, saving model to best_model.SB
Epoch 2/100
8/8 [==============================] - 0s 683us/step - loss: 1.2273

Epoch 00002: loss did not improve from 0.17939
Epoch 3/100
8/8 [==============================] - 0s 510us/step - loss: 0.9441

Epoch 00003: loss did not improve from 0.17939
Epoch 4/100
8/8 [==============================] - 0s 701us/step - loss: 0.4685

Epoch 00004: loss did not improve from 0.17939
Epoch 5/100
8/8 [==============================] - 0s 1ms/step - loss: 0.4644

Epoch 00005: loss did not improve from 0.17939
Epoch 6/100
8/8 [==============================] - 0s 1ms/step - loss: 0.6243

Epoch 00006: loss did not improve from 0.17939
Epoch 7/100
8/8 [==============================] - 0s 1ms/step - loss: 0.1885

Epoch 00007: loss did not improve from 0.17939
Epoch 8/100
8/8 [==============================] - 0s 2ms/step - loss: 0.6093



In [9]:
sgd = keras.optimizers.RMSprop(learning_rate=0.01)
model.compile(loss='mean_absolute_error',optimizer=sgd)

historyData = model.fit(xarray,df.y3,epochs=200,callbacks=[es,mc])

loss_hist = historyData.history['loss']
#The above line will return a dictionary, access it's info like this:
best_epoch = np.argmin(historyData.history['loss']) + 1
print ('best epoch = ', best_epoch)
print('smallest loss =', np.min(loss_hist))

Epoch 1/200
8/8 [==============================] - 1s 127ms/step - loss: 0.1906

Epoch 00001: loss did not improve from 0.05038
Epoch 2/200
8/8 [==============================] - 0s 496us/step - loss: 0.0599

Epoch 00002: loss did not improve from 0.05038
Epoch 3/200
8/8 [==============================] - 0s 428us/step - loss: 0.0725

Epoch 00003: loss did not improve from 0.05038
Epoch 4/200
8/8 [==============================] - 0s 531us/step - loss: 0.0443

Epoch 00004: loss improved from 0.05038 to 0.04432, saving model to best_model.SB
Epoch 5/200
8/8 [==============================] - 0s 439us/step - loss: 0.0439

Epoch 00005: loss improved from 0.04432 to 0.04387, saving model to best_model.SB
Epoch 6/200
8/8 [==============================] - 0s 1ms/step - loss: 0.0434

Epoch 00006: loss improved from 0.04387 to 0.04339, saving model to best_model.SB
Epoch 7/200
8/8 [==============================] - 0s 477us/step - loss: 0.0430

Epoch 00007: loss improved from 0.04339 to 0.043


Epoch 00057: loss did not improve from 0.02915
Epoch 58/200
8/8 [==============================] - 0s 762us/step - loss: 0.0404

Epoch 00058: loss did not improve from 0.02915
Epoch 59/200
8/8 [==============================] - 0s 1ms/step - loss: 0.0375

Epoch 00059: loss did not improve from 0.02915
Epoch 60/200
8/8 [==============================] - 0s 837us/step - loss: 0.0407

Epoch 00060: loss did not improve from 0.02915
Epoch 61/200
8/8 [==============================] - 0s 708us/step - loss: 0.0318

Epoch 00061: loss did not improve from 0.02915
Epoch 62/200
8/8 [==============================] - 0s 742us/step - loss: 0.0303

Epoch 00062: loss did not improve from 0.02915
Epoch 63/200
8/8 [==============================] - 0s 691us/step - loss: 0.0290

Epoch 00063: loss improved from 0.02915 to 0.02896, saving model to best_model.SB
Epoch 64/200
8/8 [==============================] - 0s 544us/step - loss: 0.0270

Epoch 00064: loss improved from 0.02896 to 0.02703, saving mode

8/8 [==============================] - 0s 492us/step - loss: 0.0311

Epoch 00116: loss did not improve from 0.01764
Epoch 117/200
8/8 [==============================] - 0s 684us/step - loss: 0.0315

Epoch 00117: loss did not improve from 0.01764
Epoch 118/200
8/8 [==============================] - 0s 763us/step - loss: 0.0276

Epoch 00118: loss did not improve from 0.01764
Epoch 119/200
8/8 [==============================] - 0s 411us/step - loss: 0.0323

Epoch 00119: loss did not improve from 0.01764
Epoch 120/200
8/8 [==============================] - 0s 503us/step - loss: 0.0247

Epoch 00120: loss did not improve from 0.01764
Epoch 121/200
8/8 [==============================] - 0s 478us/step - loss: 0.0227

Epoch 00121: loss did not improve from 0.01764
Epoch 122/200
8/8 [==============================] - 0s 482us/step - loss: 0.0242

Epoch 00122: loss did not improve from 0.01764
Epoch 123/200
8/8 [==============================] - 0s 463us/step - loss: 0.0239

Epoch 00123: loss did

In [10]:
sgd = keras.optimizers.RMSprop(learning_rate=0.001)
model.compile(loss='mean_absolute_error',optimizer=sgd)

historyData = model.fit(xarray,df.y3,epochs=200,callbacks=[es,mc])

loss_hist = historyData.history['loss']
#The above line will return a dictionary, access it's info like this:
best_epoch = np.argmin(historyData.history['loss']) + 1
print ('best epoch = ', best_epoch)
print('smallest loss =', np.min(loss_hist))

Epoch 1/200
8/8 [==============================] - 1s 132ms/step - loss: 0.0177

Epoch 00001: loss did not improve from 0.01764
Epoch 2/200
8/8 [==============================] - 0s 2ms/step - loss: 0.0178

Epoch 00002: loss did not improve from 0.01764
Epoch 3/200
8/8 [==============================] - 0s 3ms/step - loss: 0.0175

Epoch 00003: loss improved from 0.01764 to 0.01746, saving model to best_model.SB
Epoch 4/200
8/8 [==============================] - 0s 351us/step - loss: 0.0171

Epoch 00004: loss improved from 0.01746 to 0.01714, saving model to best_model.SB
Epoch 5/200
8/8 [==============================] - 0s 284us/step - loss: 0.0168

Epoch 00005: loss improved from 0.01714 to 0.01682, saving model to best_model.SB
Epoch 6/200
8/8 [==============================] - 0s 717us/step - loss: 0.0168

Epoch 00006: loss improved from 0.01682 to 0.01682, saving model to best_model.SB
Epoch 7/200
8/8 [==============================] - 0s 1ms/step - loss: 0.0172

Epoch 00007: loss


Epoch 00059: loss did not improve from 0.01468
Epoch 60/200
8/8 [==============================] - 0s 494us/step - loss: 0.0151

Epoch 00060: loss did not improve from 0.01468
Epoch 61/200
8/8 [==============================] - 0s 1ms/step - loss: 0.0149

Epoch 00061: loss did not improve from 0.01468
Epoch 62/200
8/8 [==============================] - 0s 569us/step - loss: 0.0152

Epoch 00062: loss did not improve from 0.01468
Epoch 63/200
8/8 [==============================] - 0s 673us/step - loss: 0.0148

Epoch 00063: loss did not improve from 0.01468
Epoch 64/200
8/8 [==============================] - 0s 427us/step - loss: 0.0153

Epoch 00064: loss did not improve from 0.01468
Epoch 65/200
8/8 [==============================] - 0s 543us/step - loss: 0.0147

Epoch 00065: loss improved from 0.01468 to 0.01465, saving model to best_model.SB
Epoch 66/200
8/8 [==============================] - 0s 366us/step - loss: 0.0145

Epoch 00066: loss improved from 0.01465 to 0.01454, saving mode

In [11]:
from __future__ import print_function
#For results of training network:

#keras.layer.get_weights() function retrieves weight values
first_layer_weights = model.layers[0].get_weights()[0]
w01 = first_layer_weights[0][0]
w02 = first_layer_weights[1][0]
w03 = first_layer_weights[2][0]
first_layer_bias  = model.layers[0].get_weights()[1]
b1 = first_layer_bias
second_layer_weights = model.layers[1].get_weights()[0]
w12 = second_layer_weights[0][0]
second_layer_bias  = model.layers[1].get_weights()[1]
b2 = second_layer_bias
third_layer_weights = model.layers[2].get_weights()[0]
w23 = third_layer_weights[0][0]
third_layer_bias  = model.layers[2].get_weights()[1]
b3 = third_layer_bias

#print weights and biases
print (first_layer_weights)
print ('w01 = ', w01, 'w02 = ', w02, 'w03 = ', w03)
print (first_layer_bias)
print ('b1 = ', b1)
print (second_layer_weights)
print ('w12 = ', w12)
print (second_layer_bias)
print ('b2 = ', b2)
print (third_layer_weights)
print ('w23 = ', w23)
print (third_layer_bias)
print ('b3 = ', b3)

#use model.predict() function to print model predictions for data conditions
xarray= np.array(xdata)
print ('x01/20.2,  x02/14.5,   x03/308.0,  y3/32.4,  a3:')
test = []
for i in range(0,8): 
    test = [[xarray[i][0], xarray[i][1], xarray[i][2]]]
    testarray = np.array(test)
    a3 = model.predict(testarray)
    print (xarray[i][0], xarray[i][1], xarray[i][2], df.y3[i], a3)
print('  ')
print ('x01,  x02,   x03,  y3,  a3*32.4:')
for i in range(0,8): 
    test = [[xarray[i][0], xarray[i][1], xarray[i][2]]]
    testarray = np.array(test)
    a3 = model.predict(testarray)
    print (xarray[i][0]*20.2, xarray[i][1]*14.5, xarray[i][2]*308.0, df.y3[i]*32.4, a3*32.4)

[[1.755288  ]
 [0.18251595]
 [0.63163596]]
w01 =  1.755288 w02 =  0.18251595 w03 =  0.63163596
[-0.5691421]
b1 =  [-0.5691421]
[[0.76431924]]
w12 =  0.76431924
[-0.23482344]
b2 =  [-0.23482344]
[[0.43096396]]
w23 =  0.43096396
[0.41481322]
b3 =  [0.41481322]
x01/20.2,  x02/14.5,   x03/308.0,  y3/32.4,  a3:
0.9900990099009901 0.896551724137931 1.009090909090909 0.9558346964599856 [[0.9624466]]
0.9900990099009901 1.0 1.0 0.9968828122588808 [[0.9667744]]
0.9900990099009901 1.0551724137931036 0.9935064935064936 0.9721922162896206 [[0.96874034]]
1.0 0.896551724137931 1.009090909090909 0.9539829017622912 [[0.9681711]]
0.9900990099009901 1.0 1.0 1.003055461251196 [[0.9667744]]
1.0 1.0551724137931036 0.9935064935064936 0.9691058917934631 [[0.9744649]]
1.188118811881188 0.896551724137931 1.009090909090909 1.0984228881824636 [[1.0769379]]
1.7821782178217822 1.0 1.0 1.4320545662170918 [[1.4247398]]
  
x01,  x02,   x03,  y3,  a3*32.4:
20.0 13.0 310.8 30.969044165303533 [[31.18327]]
20.0 14.5 308.0